In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
size = 'dev_s'
prob = 50
splitsize = 60

In [ ]:
import pandas as pd

from utils import filename
from utils.data.segmentmap import create_from_files


segmentmap, allocation_dict = create_from_files(size, regenerate=False)
df = pd.read_csv(filename.data.true(size), sep=' ')
fname = filename.data.sky(size)

In [ ]:
from utils.data.generating import split_by_size
from definitions import config

spatial = config['segmentation']['cube_size']['spatial']
freq = config['segmentation']['cube_size']['freq']
cube_dim = (spatial, spatial, freq)
empty_cube_dim = tuple([c * 2 for c in cube_dim])

galax_prob = prob / 100

splitted_datasets = split_by_size(df, fname, segmentmap, allocation_dict, galax_prob, cube_dim, empty_cube_dim, n_memory_batches=50, splitsize=splitsize)

In [ ]:
import torch
from tqdm.auto import tqdm
import string
import random
import glob

directory = filename.processed.dataset(size, prob)

for f in glob.glob('{}/*'.format(directory)):
    os.remove(f)

for i, dataset_split in enumerate(tqdm(splitted_datasets)):
    name = ''.join(random.choice(string.ascii_letters) for i in range(20))
    torch.save(dataset_split, directory + '/{}.pt'.format(name))